In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-03-26 09:08:47.865817: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 09:08:47.890833: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 09:08:47.890857: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 09:08:47.890881: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 09:08:47.896521: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-26 09:08:49.677971: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 09:08:49.704113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 09:08:49.704322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))
bird_classes
test_dataset_224 = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)
test_dataset_299= tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(299, 299),
    shuffle=False,
    seed=123,

)

Found 11545 files belonging to 167 classes.


2024-03-26 09:08:49.988704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 09:08:49.988926: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 09:08:49.989088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 11545 files belonging to 167 classes.


In [3]:
preprocess_input_effnet = tf.keras.applications.efficientnet.preprocess_input
preprocess_input_effnet_v2 = tf.keras.applications.efficientnet_v2.preprocess_input
preprocess_input_mobilenetsmall = tf.keras.applications.mobilenet.preprocess_input
preprocess_input_mobilenetv3largee = tf.keras.applications.mobilenet_v3.preprocess_input
preprocess_input_resnet = tf.keras.applications.resnet.preprocess_input

In [22]:
test_dataset_224_effb7 = test_dataset_224.map(lambda x, y: (preprocess_input_effnet(x), y))
test_dataset_224_effV2L = test_dataset_224.map(lambda x, y: (preprocess_input_effnet_v2(x), y))
test_dataset_224_mobilenetsmall = test_dataset_224.map(lambda x, y: (preprocess_input_mobilenetsmall(x), y))
test_dataset_224_mobilenetv3large= test_dataset_224.map(lambda x, y: (preprocess_input_mobilenetv3largee(x), y))
test_dataset_224_resnet = test_dataset_224.map(lambda x, y: (preprocess_input_resnet(x), y))

In [5]:
effb7 = tf.keras.models.load_model("Best_models/best_eff_b7_90.67")
effv2l = tf.keras.models.load_model("Best_models/best_effnetv2L_90.57")
mobilenetsmall = tf.keras.models.load_model("Best_models/best_mobilenet_small_89.65")
mobilenetlarge = tf.keras.models.load_model("Best_models/best_mobilenetv3large_90.67")
resnet = tf.keras.models.load_model("Best_models/resnet_90.22")


In [7]:
effb7_pred = effb7.predict(test_dataset_224_effb7)
effb7_pred

361/361 [==============================] - 45s 126ms/step


array([[5.33957422e-01, 4.05637060e-25, 5.59847724e-11, ...,
        4.79629873e-19, 2.13901688e-14, 3.73075451e-23],
       [5.18665612e-01, 1.70382812e-14, 6.37755932e-07, ...,
        5.78528718e-11, 2.95757885e-09, 1.14885640e-12],
       [5.12335122e-01, 1.66763383e-25, 3.38355544e-14, ...,
        5.15714574e-20, 7.28958113e-14, 9.07663989e-24],
       ...,
       [1.34281084e-17, 3.67159618e-13, 4.65406706e-11, ...,
        2.13902005e-19, 2.98250799e-11, 9.99867082e-01],
       [0.00000000e+00, 2.97335929e-35, 2.23934658e-35, ...,
        0.00000000e+00, 2.54032928e-36, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]], dtype=float32)

In [12]:
effb7_pred = effb7_pred.argmax(axis=1)

In [13]:
effv2l_pred = effv2l.predict(test_dataset_224_effV2L).argmax(axis=1)
effv2l_pred

361/361 [==============================] - 41s 105ms/step


array([  0,   0,   0, ...,   2, 166, 166])

In [14]:
mobilenetsmall_pred = mobilenetsmall.predict(test_dataset_224_mobilenetsmall).argmax(axis=1)
mobilenetsmall_pred

361/361 [==============================] - 6s 14ms/step


array([106, 106, 106, ..., 106, 106, 106])

In [15]:
mobilenetlarge_pred = effv2l.predict(test_dataset_224_mobilenetv3large).argmax(axis=1)
mobilenetlarge_pred

361/361 [==============================] - 36s 101ms/step


array([  0,   0,   0, ...,   2, 166, 166])

In [23]:
resnet_pred = resnet.predict(test_dataset_224_resnet).argmax(axis=1)
resnet_pred

361/361 [==============================] - 24s 65ms/step


array([  0,   0,  27, ..., 166, 166, 166])

In [26]:
import pandas as pd
effv2l_pred = pd.DataFrame(effv2l_pred)
resnet_pred = pd.DataFrame(resnet_pred)
mobilenetsmall_pred = pd.DataFrame(mobilenetsmall_pred)
mobilenetlarge_pred = pd.DataFrame(mobilenetlarge_pred)
effb7_pred = pd.DataFrame(effb7_pred)

In [29]:
results = pd.concat([effb7_pred, effv2l_pred, mobilenetlarge_pred, mobilenetsmall_pred, resnet_pred], axis=1)
results

,0,0,0,0,0
0,0,0,0,106,0
1,0,0,0,106,0
2,0,0,0,106,27
3,0,0,0,106,0
4,0,0,0,106,27
...,...,...,...,...,...
11540,166,166,166,106,166
11541,166,155,155,106,166
11542,166,2,2,106,166
11543,166,166,166,106,166


In [31]:
results['max_count']= results.apply(lambda row: row.mode().iloc[0], axis=1)
results

,0,0,0,0,0,max_count
0,0,0,0,106,0,0
1,0,0,0,106,0,0
2,0,0,0,106,27,0
3,0,0,0,106,0,0
4,0,0,0,106,27,0
...,...,...,...,...,...,...
11540,166,166,166,106,166,166
11541,166,155,155,106,166,155
11542,166,2,2,106,166,2
11543,166,166,166,106,166,166


In [77]:
max_count_res = results[['max_count']].copy()
max_count_res = np.asarray(max_count_res)

,max_count
0,0
1,0
2,0
3,0
4,0
...,...
11540,166
11541,155
11542,2
11543,166


In [35]:
y_labels = []
for images, labels in test_dataset_224_resnet.unbatch():
    y_labels.append(labels.numpy())
import numpy as np
y_labels = np.asarray(y_labels)
y_labels = y_labels.argmax(axis=1)

AttributeError: 'list' object has no attribute 'dtype'

In [76]:
mapping = {27:0, 37:1, 55:2, 58:3, 88:4,93:5, 97:6, 111:7, 120:8, 130:9, 138:10, 149:11}

In [90]:
yLabels_df = pd.DataFrame(y_labels)
max_count_df = pd.DataFrame(max_count_res)
max_count_df = max_count_df.rename(columns={0:1})
sum = pd.concat([yLabels_df, max_count_df],axis=1 )
sum

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
11540,166,166
11541,166,155
11542,166,2
11543,166,166


In [91]:
def compare_values(row):
    val1 = row[0]  # Assuming the first column is the one to be compared
    val2 = row[1]  # Assuming the second column is the one to be compared
    
    if val1 == val2:
        return 1
    elif val1 in mapping and mapping[val1] == val2:
        return 1
    else:
        return 0
sum_df = pd.DataFrame(sum)
# Apply the custom function to each row and create a new column 'Count'
sum_df['Count'] = sum_df.apply(compare_values, axis=1)

print(sum_df)

         0    1  Count
0        0    0      1
1        0    0      1
2        0    0      1
3        0    0      1
4        0    0      1
...    ...  ...    ...
11540  166  166      1
11541  166  155      0
11542  166    2      0
11543  166  166      1
11544  166  166      1

[11545 rows x 3 columns]


In [95]:
(11023/11545) * 100

95.47856214811607